In [1]:
import os

import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd

from numpy import asarray
from numpy import savetxt
from os.path import join
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

import audio_training
import audio_input
import audio_create_model


In [3]:
training_folder = '../data/vid/trainingSet/raw'
audio_create_model.create_model(training_folder)
audio_input.audio_input('userData/audioInput/userInput.wav')